<a href="https://colab.research.google.com/github/rlaaudrb1104/Ai/blob/PJH/graphcodebert_CWE_%EB%8B%A4%EC%A4%91_%EB%AA%A8%EB%8D%B8_%ED%99%95%EB%A5%A0_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install datasets
!pip install torch
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 19.1 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py

In [6]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import re

# 모델 및 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
models = {
    "cwe-119": AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/models/CWE_models/CWE-119_vul"),
    "cwe-20": AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/models/CWE_models/CWE-20_vul"),
    "cwe-125": AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/models/CWE_models/CWE-125_vul"),
    "cwe-787": AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/models/CWE_models/CWE-787_vul")
}

def preprocess2(code_text):
    # 주석 제거
    code_text = re.sub(r'//.*?\n', '', code_text, flags=re.DOTALL)
    code_text = re.sub(r'/\*.*?\*/', '', code_text, flags=re.MULTILINE)
    code_text = re.sub(r'^#define\s+<.*?>\s*\n', '', code_text, flags=re.MULTILINE)
    code_text = re.sub(r'#include .*?\n', '', code_text, flags=re.DOTALL)
    code_text = re.sub(r'#include "*.*?\n', '', code_text, flags=re.DOTALL)
    code_text = re.sub(r'#define .*?\n', '', code_text, flags=re.DOTALL)
    code_text = re.sub(r'#undef\s+\w+', '', code_text, flags=re.DOTALL)
    code_text = re.sub(r'#if\s+\w+', '', code_text, flags=re.DOTALL)
    code_text = re.sub(r'#else\s+\w+', '', code_text, flags=re.DOTALL)
    code_text = re.sub(r'#elif\s+\w+', '', code_text, flags=re.DOTALL)
    code_text = re.sub(r'#endif', '', code_text, flags=re.DOTALL)
    code_text = re.sub(r'[\t ]+', '', code_text, flags=re.DOTALL)
    code_text = re.sub(r'\n\s*\n', '', code_text, flags=re.DOTALL)
    code_text = re.sub(r'\n', '', code_text, flags=re.DOTALL)
    code_text = re.sub(r'return*.*?;', '', code_text, flags=re.DOTALL)
    code_text = re.sub(r'return;', '', code_text, flags=re.DOTALL)

    # 코드 내의 공백 및 줄 바꿈 제거
    code_text = re.sub(r'\s+', ' ', code_text)

    return code_text

def predict_vulnerability(input_code, models, tokenizer):
    # 입력 코드 전처리
    preprocessed_text = preprocess2(input_code)

    results = {}

    for model_name, model in models.items():
        # 입력 토크나이징
        inputs = tokenizer(preprocessed_text, return_tensors="pt", padding=True, truncation=True, max_length=512)

        # 모델 예측
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            probs = torch.sigmoid(logits).squeeze()  # 시그모이드 함수로 확률 계산

            # 취약점 발생 확률 (레이블 1) 예측
            vulnerability_probability = probs[1].item()  # 레이블 1의 확률

            # 결과 저장
            results[model_name] = {
                "vulnerability_probability": vulnerability_probability
            }

    # 가장 높은 확률의 모델 선택
    best_model = max(results, key=lambda k: results[k]["vulnerability_probability"])

    return results, best_model

# 입력 코드 예시
input_code = """
void host_lookup(char *user_supplied_addr){
struct hostent *hp;
in_addr_t *addr;
char hostname[64];
in_addr_t inet_addr(const char *cp);

/*routine that ensures user_supplied_addr is in the right format for conversion */

validate_addr_form(user_supplied_addr);
addr = inet_addr(user_supplied_addr);
hp = gethostbyaddr( addr, sizeof(struct in_addr), AF_INET);
strcpy(hostname, hp->h_name);
}
"""

# 취약점 예측 실행
predictions, best_model = predict_vulnerability(input_code, models, tokenizer)

# 결과 출력
for model_name, result in predictions.items():
    vulnerability_probability = result["vulnerability_probability"]
    vulnerability_status = "취약점 존재" if vulnerability_probability >= 0.5 else "취약점 없음"
    print(f"{model_name}: {vulnerability_status}, Probability: {vulnerability_probability:.4f}")

if best_model:
    print(f"\n가장 높은 확률을 가진 모델: {best_model}")


cwe-119: 취약점 없음, Probability: 0.1700
cwe-20: 취약점 없음, Probability: 0.1987
cwe-125: 취약점 없음, Probability: 0.1622
cwe-787: 취약점 없음, Probability: 0.0952

가장 높은 확률을 가진 모델: cwe-20
